In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
import os
import datetime

from utils.config import get_gpu_memory
from utils.config import get_callbacks
from models.train import train_model
from utils.visualize import plot_training
from utils.visualize import plot_confusion_matrix
from utils.visualize import plot_roc_curve
from utils.evaluate import evaluate_model
from utils.evaluate import calculate_metrics

from models.model import Models

In [ ]:
# Verifica se há GPU disponível
get_gpu_memory()

# Configurações
IMG_SIZE = (224, 224)  # Tamanho da imagem
BATCH_SIZE = 128
EPOCHS = 100        # Número de épocas
NUM_CLASSES = 2     # Número de classes
PACIENTE = 7        # Paciência para o EarlyStopping
DELTA = 0.001       # Delta para o ReduceLROnPlateau

BASE_PATH = "/mnt/efs-tcga/HEAL_Workspace/macenko_datas/"

# Caminhos para as imagens 
PATH_IMGS = BASE_PATH + 'splited'

MODEL_NAME = "denseNet121" 
LOG_DIR =  f"logs/fit/{MODEL_NAME}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


WEIGHT_PATH = BASE_PATH + "weights/"
if not os.path.exists(WEIGHT_PATH):
    os.makedirs(WEIGHT_PATH)


CHECKPOINT_PATH = BASE_PATH + "weights/checkpoint/"
if not os.path.exists(CHECKPOINT_PATH):
    os.makedirs(CHECKPOINT_PATH)


CHECKPOINT_ALL_PATH = BASE_PATH + "weights/checkpoint_all/"
if not os.path.exists(CHECKPOINT_ALL_PATH):
    os.makedirs(CHECKPOINT_ALL_PATH)


MODEL_PATH = "/home/ec2-user/SageMaker/" + "models/"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

# Configurações de avaliação
EVAL_BATCH_SIZE = 64
EVAL_PATH = BASE_PATH + "splited/test/"
if not os.path.exists(EVAL_PATH):
    os.makedirs(EVAL_PATH)

# Configurações de teste
TEST_BATCH_SIZE = 64
TEST_PATH = BASE_PATH + "splited/test/"
if not os.path.exists(TEST_PATH):
    os.makedirs(TEST_PATH)

# Configurações de predição
PRED_BATCH_SIZE = 64
PRED_PATH = BASE_PATH + "splited/test/"
if not os.path.exists(PRED_PATH):
    os.makedirs(PRED_PATH)

# Configurações de visualização
VISUALIZE_BATCH_SIZE = 64
VISUALIZE_PATH = BASE_PATH + "splited/test/"
if not os.path.exists(VISUALIZE_PATH):
    os.makedirs(VISUALIZE_PATH)


md = Models()

In [ ]:
train_generator, validation_generator, test_generator = md.get_generators(PATH_IMGS, IMG_SIZE, BATCH_SIZE)

In [ ]:
# Criar modelo DenseNet dentro da estratégia
list_device = None
strategy = md.get_strategy(list_device)
with strategy.scope():
    # Criando e compilando o modelo
    model = md.create_densenet_model(pretrained=True, num_classes=NUM_CLASSES, img_size=IMG_SIZE)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.summary()



In [ ]:
# Callbacks
early_stopping, checkpoint, checkpoint_all, tensorboard_callback = get_callbacks(model_name=MODEL_NAME,
                                                                                 checkpoint_path=CHECKPOINT_PATH, 
                                                                                 checkpoint_all_path=CHECKPOINT_ALL_PATH, 
                                                                                 delta=DELTA, 
                                                                                 pacience=PACIENTE, 
                                                                                 log_dir=LOG_DIR,
                                                                                 save_model_per_epoch=True)

In [ ]:
# Treinamento
history = train_model(  model=model, 
                        model_name=MODEL_NAME,
                        model_path=MODEL_PATH, 
                        weights_path=WEIGHT_PATH, 
                        batch_size=BATCH_SIZE, 
                        epochs=EPOCHS, 
                        early_stopping=early_stopping, 
                        checkpoint=checkpoint, 
                        checkpoint_all=checkpoint_all,
                        tensorboard_callback=tensorboard_callback, 
                        train_generator=train_generator, 
                        val_generator=validation_generator,
                        initial_epoch=0,
                        load_weight=None
                    )

In [ ]:
# Plotar evolução do treinamento
plot_training(history)

# Exibir instruções para visualizar o TensorBoard
print("\nPara visualizar o treinamento em tempo real, execute no terminal:")
print("tensorboard --logdir=logs/fit/{MODEL_NAME}")
print("Depois, abra no navegador: http://localhost:6006")


## **Avaliando o Modelo**

In [ ]:
# Caminho do modelo salvo
tf.keras.backend.clear_session()
model_path = BASE_PATH + "weights/checkpoint_all/" + f"{MODEL_NAME}_epoch_10.keras"

# Carregar modelo
model = load_model(model_path, compile=False)
class_indices = test_generator.class_indices  # Índices das classes
class_labels = list(class_indices.keys())  # Nomes das classes

# Avaliação no conjunto de teste
y_true = test_generator.classes  # Classes verdadeiras
y_pred_probs = model.predict(test_generator)  # Probabilidades preditas
y_pred = np.argmax(y_pred_probs, axis=1)  # Classes preditas

# Calcular métricas
calculate_metrics(y_true, y_pred, class_labels, MODEL_NAME)

# Plotar matriz de confusão
plot_confusion_matrix(y_true, y_pred, class_labels, MODEL_NAME)

# Plotar curva ROC
plot_roc_curve(y_true, y_pred_probs, class_labels, MODEL_NAME)


## **Treinamento com mais épocas**

In [ ]:
# carregar o modelo salvo
INITIAL_EPOCH = 10  # Mude conforme necessário
model = tf.keras.models.load_model(BASE_PATH + "weights/checkpoint_all/" + f'{MODEL_NAME}_epoch_{INITIAL_EPOCH}.keras')
load_weight = BASE_PATH + "weights/checkpoint_all/" + f"{MODEL_NAME}_epoch_{INITIAL_EPOCH}.keras"

# Verifica se há GPU disponível
get_gpu_memory()

# Configurações
IMG_SIZE = (224, 224)  # Tamanho da imagem
BATCH_SIZE = 128
EPOCHS = 100        # Número de épocas
NUM_CLASSES = 2     # Número de classes
PACIENTE = 15        # Paciência para o EarlyStopping
DELTA = 0.001       # Delta para o ReduceLROnPlateau


In [ ]:
train_generator, validation_generator, test_generator = md.get_generators(PATH_IMGS, IMG_SIZE, BATCH_SIZE)

In [ ]:
# Criar modelo DenseNet dentro da estratégia
list_device = None
strategy = md.get_strategy(list_device)
with strategy.scope():
    # Criando e compilando o modelo
    model = md.create_densenet_model(pretrained=True, num_classes=NUM_CLASSES, img_size=IMG_SIZE)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# Callbacks
early_stopping, checkpoint, checkpoint_all, tensorboard_callback = get_callbacks(model_name=MODEL_NAME,
                                                                                 checkpoint_path=CHECKPOINT_PATH, 
                                                                                 checkpoint_all_path=CHECKPOINT_ALL_PATH, 
                                                                                 delta=DELTA, 
                                                                                 pacience=PACIENTE, 
                                                                                 log_dir=LOG_DIR,
                                                                                 save_model_per_epoch=True)

In [ ]:
# Treinamento
history = train_model(
                        model=model, 
                        model_name=MODEL_NAME,
                        model_path=MODEL_PATH, 
                        weights_path=WEIGHT_PATH, 
                        batch_size=BATCH_SIZE, 
                        epochs=EPOCHS, 
                        early_stopping=early_stopping, 
                        checkpoint=checkpoint, 
                        checkpoint_all=checkpoint_all,
                        tensorboard_callback=tensorboard_callback, 
                        train_generator=train_generator, 
                        val_generator=validation_generator,
                        initial_epoch=INITIAL_EPOCH,
                        load_weight=load_weight
                    )

In [ ]:
# Plotar evolução do treinamento
plot_training(history, MODEL_NAME)

# Exibir instruções para visualizar o TensorBoard
print("\nPara visualizar o treinamento em tempo real, execute no terminal:")
print(f"tensorboard --logdir=logs/fit/{MODEL_NAME}")
print("Depois, abra no navegador: http://localhost:6006")


## **Avaliando o Modelo**

In [ ]:
# Caminho do modelo salvo
model_path = BASE_PATH + "/models" + f"{MODEL_NAME}_model_224x224.keras"

# Carregar modelo
model = load_model(model_path, compile=False)
class_indices = test_generator.class_indices  # Índices das classes
class_labels = list(class_indices.keys())  # Nomes das classes

# Avaliação no conjunto de teste
y_true = test_generator.classes  # Classes verdadeiras
y_pred_probs = model.predict(test_generator)  # Probabilidades preditas
y_pred = np.argmax(y_pred_probs, axis=1)  # Classes preditas

# Calcular métricas
calculate_metrics(y_true, y_pred, class_labels, MODEL_NAME)

# Plotar matriz de confusão
plot_confusion_matrix(y_true, y_pred, class_labels, MODEL_NAME)

# Plotar curva ROC
plot_roc_curve(y_true, y_pred_probs, class_labels, MODEL_NAME)
